In [5]:
# Basic definitions and imports

using Zygote
using QuantumInformation, LinearAlgebra
using Plots, Printf; pyplot()
using LazySets, Polyhedra
default(:size, (1200,800))

macro printvals(syms...)
  l = length(syms)
  quote
    $([(i == l) ? :(print(String($(Meta.quot(syms[i]))) * " = "); println($(syms[i]))) :
       :(print(String($(Meta.quot(syms[i]))) * " = "); print($(syms[i])); print(", ")) for i in eachindex(syms)]...)
  end |> esc
end

macro sliceup(arr, info...)
  l = Integer(length(info)/2)
  names = Vector{Any}(undef, l)
  steps = Vector{Any}(undef, l)
  for i in eachindex(names)
    names[i] = info[2*i-1]
    steps[i] = info[2*i]
  end

  quote
    curr = 1
    $([:(next = curr + $(steps[j]) - 1; $(names[j]) = $arr[curr:next]; curr = next + 1) for j in 1:l]...)
  end |> esc
end

@sliceup (macro with 1 method)

In [6]:
# Pironio rates

h(x) = -x*log2(x) - (1-x)*log2(1-x)
r(Q,S) = 1-h((1+sqrt((S/2)^2 - 1))/2)-h(Q)

function plot_pironio(Qval=nothing, Sval=nothing, samples=100)
  if isnothing(Qval) && isnothing(Sval)
    plot(range(0,stop=0.5,length=samples), range(2,stop=2*sqrt(2),length=samples), r, st=:surface, xlabel="Q",ylabel="S")
  elseif isnothing(Sval)
    drdS = S -> gradient((Q, S) -> r(Q,S), Qval, S)[2]
    plot(range(2,stop=2*sqrt(2),length=samples), drdS, xlabel="S",ylabel="dr/dS", label="Q = $Qval")
  else
    drdQ = Q -> gradient((Q, S) -> r(Q,S), Q, Sval)[1]
    plot(range(0,stop=0.5,length=samples), drdQ, xlabel="Q",ylabel="dr/dQ", label="S = $Sval")
  end
end

plot_pironio (generic function with 4 methods)

In [7]:
# Asymmetric CHSH

phi(x) = h(0.5 + 0.5*x)
g(q, alpha, s) = 1 + phi(sqrt((1-2*q)^2 +4*q*(1-q)*((s^2/4)-alpha^2))) - phi((s^2/4)-alpha^2)
sstar(q, alpha) = 2 # TODO
function gbar(q, alpha, s)
  if abs(alpha) < 1 && s < sstar(q, alpha)
    return g(q, alpha, s) # TODO
  else
    return g(q, alpha, s)
  end
end

gchsh(s) = 1-phi(sqrt(s^2/4 - 1))

gchsh (generic function with 1 method)

In [8]:
# Wirings

num_wirings(c, o, i) = o^(i^c * o^c) * prod([i^(i^(j-1) * o^(j-1)) for j in 1:c])
function wiring_prob(CA, CAi, CB, CBi, pabxy, pax, pby)
  oA, oB, iA, iB = size(pabxy)
  ppabxy, ppax, ppby = (zeros(size(p)) for p in [pabxy, pax, pby]) |> collect
  c = Integer((CA |> size |> length) / 2)

  # iterate over every possible combination
  for params in Iterators.product(vcat([[1:n for i in 1:c] for n in [oA, oB, iA, iB]]...)...)

  @sliceup params as oA bs oB xs iA ys iB

  pABXY = 1
  pBY = 1
  pAX = 1
  for i in 1:c
    if CAi[i, as[1:i-1]..., xs[1:i-1]...] != xs[i] || CBi[i, bs[1:i-1]..., ys[1:i-1]...] != ys[i]
      pABXY = 0
      pAX = 0
      pBY = 0
      break
    else
      pABXY *= pabxy[as[i], bs[i], xs[i], ys[i]]
      pAX *= pax[as[i], xs[i]]
      pBY *= pby[bs[i], ys[i]]
    end
  end

  xp = CA[as..., xs...]
  yp = CB[bs..., ys...]
  ppabxy[as[1], bs[1], xp, yp] += pABXY
  ppax[as[1], xp] += pAX
  ppby[bs[1], yp] += pBY
  end

  return ppabxy, ppax, ppby
end

wiring_prob (generic function with 1 method)

In [11]:
# Koon Tong's model

kd(i,j) = (i == j) ? 1 : 0
E(M, rho) = tr(M * rho) 
sigmas = [[kd(j, 3) kd(j,1)-im*kd(j,2); kd(j,1)+im*kd(j,2) -kd(j,3)] for j in 1:3]
psi(theta) = cos(theta) * kron(ket(1,2), ket(1,2)) + sin(theta) * kron(ket(2,2), ket(2,2))
rho(theta) = proj(psi(theta))
Mtld(mu) = cos(mu) * sigmas[3] + sin(mu) * sigmas[1]

# let outcome 2 be associated with eigenvalue -1
function expt_model_probs(pabxy, pax, pby, etaA, etaB, nc)
  oA, oB, iA, iB = size(pabxy)
  ppabxy, ppax, ppby = (zeros(size(p)) for p in [pabxy, pax, pby]) |> collect

  for a in 1:oA, b in 1:oB, x in 1:iA, y in 1:iB
    ppax[a,x] = kd(a,-1)*(nc+1-etaA) + (etaA-nc)*p[a,x]
    ppby[b,y] = kd(b,-1)*(nc+1-etaB) + (etaB-nc)*p[b,y]
    ppabxy[a,b,x,y] = nc * kd(a,-1) * kd(b,-1) + (1-nc) * (kd(a,-1)*kd(b,-1)*(1-etaA*etaB))
    + etaA*etaB*pabxy[a,b,x,y] + kd(a,-1)*etaB*(1-etaA)*pax[a,x] + kd(b,-1)*etaA*(1-etaB)*pby[b,y]
  end

  return ppabxy, ppax, ppby
end

# TODO generalise to other ways to bound H(A|E)
function entropy_data(ncs, etas; theta=0.15*pi, mus=[pi, 2.53*pi], nus=[2.8*pi, 1.23*pi, pi])
  oA, oB = 2,2
  iA = length(mus); iB = length(nus) # mus for A, nus for B
  rhov = rho(theta)
  rhoA = ptrace(rhov, [2,2], 2)
  rhoB = ptrace(rhov, [2,2], 1)

  Atlds = [E(Mtld(mu), rhoA) for mu in mus]
  Btlds = [E(Mtld(nu), rhoB) for nu in nus]
  ABtlds = [E(kron(Mtld(mu), Mtld(nu)), rhov) for mu in mus, nu in nus]

  ncl = length(ncs); etal = length(etas)
  pts = Array{Tuple{Float64, Float64}, 2}(undef, ncl, etal)
  grads = Array{Tuple{Float64, Float64}, 2}(undef, ncl, etal)

  for nci in eachindex(ncs), etai in eachindex(etas)
    nc = ncs[nci]; eta = etas[etai]
    Eax = Float64[-nc-(1-nc)*((1-eta)-eta*Atlds[x]) for x in 1:iA]
    Eby = Float64[-nc-(1-nc)*((1-eta)-eta*Btlds[y]) for y in 1:iB]
    Eabxy = Float64[nc + (1-nc)*(eta^2 * ABtlds[x,y] - eta*(1-eta)*(Atlds[x] + Btlds[y]) + (1-eta)^2) for x in 1:iA, y in 1:iB]

    gradeta = Float64[ (1-nc)*((1-2*eta)*(Atlds[x] + Btlds[y]) - 2*eta*ABtlds[x,y] + 2 - 2*eta) for x in 1:iA, y in 1:iB ] 
    gradnc = Float64[ eta*((1-eta)*(Atlds[x] + Btlds[y]) - eta*ABtlds[x,y] + 2 - eta) for x in 1:iA, y in 1:iB ]

    # non-generic part below
    Q = (1 - Eabxy[1,3]) / 2 # QBER H(A|B)
    S = Eabxy[1,1] + Eabxy[1,2] + Eabxy[2,1] - Eabxy[2,2]
    if abs(S) < 2
      S = sign(S) * 2
    end
    pts[nci, etai] = (Q, gchsh(S))

    Qgradnc = - gradnc[1,3]
    if !isfinite(Qgradnc)
      Qgradnc = 0
    end
    Sgradnc = gradnc[1,1] + gradnc[1,2] + gradnc[2,1] - gradnc[2,2]
    if !isfinite(Sgradnc)
      Sgradnc = 0
    end
    HgradS = S/(4*sqrt(S^2-4)) * log2( (2+sqrt(S^2-4)) / (2-sqrt(S^2-4)) )
    if !isfinite(HgradS)
      HgradS = 0
    end
    Hgradnc = Sgradnc * HgradS
    grads[nci, etai] = (Qgradnc, Hgradnc)
  end

  return pts, grads
end

function entropy_plot(; theta=0.15*pi, mus=[pi, 2.53*pi], nus=[2.8*pi, 1.23*pi, pi], ncsamples=20, etasamples=5, etastart=0.8)
  ncs = range(0, stop=1, length=ncsamples)
  etas = range(etastart, stop=1, length=etasamples)

  qSreg, qSreggrad = entropy_data(ncs, etas, theta=theta, mus=mus, nus=nus)

  Hmaxs = map(max, qSreg...)
  Hmins = map(min, qSreg...)
  Hranges = [Hmaxs[i] - Hmins[i] for i in 1:2]
  Hlims = [(Hmins[i] - Hranges[i]*0.1, Hmaxs[i] + Hranges[i]*0.1) for i in 1:2]

  etas_const_nc = range(0.8, stop=1, length=200)
  const_nc, const_nc_grad = entropy_data([0.0], etas_const_nc, theta=theta, mus=mus, nus=nus)

  # plot qSs/qQs on plot of H(A|E) against H(A|B)
  plt = plot(range(0,stop=1,length=100), range(0,stop=1,length=100), xlabel="H(A|B)",ylabel="H(A|E)", label="Devetak-Winter bound", xlims=Hlims[1], ylims=Hlims[2])

  # find S region
  for etai in 1:etasamples
    plot!(plt, qSreg[:, etai], label="eta = $(etas[etai])")
  end
  plot!(plt, vec(const_nc), label="nc = 0")

  quivlens = 0.1 .* Hranges
  quivpts = vcat(qSreg |> vec, const_nc |> vec)
  rawgrads = vcat(qSreggrad |> vec, const_nc_grad |> vec)
  grads = [Tuple([grad[i] * quivlens[i] for i in 1:2]) for grad in rawgrads] 
  quiver!(plt, quivpts, quiver=grads)

  # TODO find Sa regions

  return plt
end

entropy_plot (generic function with 1 method)

In [14]:
using Interact
entrplot = @manipulate for theta=0:0.05:2*pi, muA1=0:0.05:2*pi, muA2=0:0.05:2*pi, nuB1=0:0.05:2*pi, nuB2=0:0.05:2*pi, nuB3=0:0.05:2*pi, ncsamples=20, etasamples=10, etastart=0:0.05:1
  mus = [muA1, muA2]
  nus = [nuB1, nuB2, nuB3]
  vbox(entropy_plot(theta=theta, mus=mus, nus=nus, ncsamples=ncsamples, etasamples=etasamples, etastart=etastart))
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["theta"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:max => 126, :min => 1, :attributes => Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal"), :step => 1, :className => "slider slider is-fullwidth", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{Observables.AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing), "index" => (Observable{Any} with 2 listeners. Value:
63, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/home/aquohn/.julia/packages/Knockout/IP1uR/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/home/aquohn/.julia/packages/Knockout/IP1uR/src/../assets/knockout_punches.js"), Asset("js", nothing, "/home/aquohn/.julia/packages/InteractBase/Zfu5P/src/../assets/all.js"), Asset("css", nothing, "/home/aquohn/.julia/packages/InteractBase/Zfu5P/src/../assets/style.css"), Asset("css", nothing, "/home/aquohn/.julia/packages/Interact/pVyN2/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00007f4143764e80, Task (runnable) @0x00007f4143764e80), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"0.0\",\"0.05\",\"0.1\",\"0.15\",\"0.2\",\"0.25\",\"0.3\",\"0.35\",\"0.4\",\"0.45\",\"0.5\",\"0.55\",\"0.6\",\"0.65\",\"0.7\",\"0.75\",\"0.8\",\"0.85\",\"0.9\",\"0.95\",\"1.0\",\"1.05\",\"1.1\",\"1.15\",\"1.2\",\"1.25\",\"1.3\",\"1.35\",\"1.4\",\"1.45\",\"1.5\",\"1.55\",

In [16]:
freqs = OrderedDict(zip(["pi/4", "π/2", "3π/4", "π"], [π/4, π/2, 3π/4, π]))
x = 0:0.1:30
mp = @manipulate for freq1 in freqs, freq2 in slider(0.01:0.1:4π; label="freq2")
    y = @. sin(freq1*x) * sin(freq2*x)
    plot(x, y)
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["freq1"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :span), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text : key")))], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "click: function (){\$root.update(val)}, css: {'is-primary is-selected' : \$root.index() == val, 'interact-widget is-medium button ' : true}")))], Dict{Symbol, Any}(:attributes => ("data-bind" => "foreach : options_js"), :className => "buttons has-addons is-centered"))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[], Dict{Symbol, Any}())], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{Observables.AbstractObservable, Union{Nothing, Bool}}}("options_js" => (Observable{Any} with 1 listeners. Value:
OrderedDict[OrderedDict{String, Any}("key" => "pi/4", "val" => 1, "id" => "idRazq0t0o"), OrderedDict{String, Any}("key" => "π/2", "val" => 2, "id" => "idXsgJbiBe"), OrderedDict{String, Any}("key" => "3π/4", "val" => 3, "id" => "idHVGh3zkP"), OrderedDict{String, Any}("key" => "π", "val" => 4, "id" => "idAEuNuvql")], nothing), "index" => (Observable{Any} with 2 listeners. Value:
1, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/home/aquohn/.julia/packages/Knockout/IP1uR/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/home/aquohn/.julia/packages/Knockout/IP1uR/src/../assets/knockout_punches.js"), Asset("js", nothing, "/home/aquohn/.julia/packages/InteractBase/Zfu5P/src/../assets/all.js"), Asset("css", nothing, "/home/aquohn/.julia/packages/InteractBase/Zfu5P/src/../assets/style.css"), Asset("css", nothing, "/home/aquohn/.julia/packages/Interact/pVyN2/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("options_js" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"options_js\"]()) ? (this.valueFromJulia[\"options_js\"]=true, this.model[\"options_js\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00007f4143632230, Task (runnable) @0x00007f4143632230), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBi